In [1]:
import pandas as pd
import statistics

import sklearn.metrics as metrics
import fca as fca

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# Creación de un retículo a partir del valor de la mediana
Hacemos un retículo a partir de un modelo que simplifique los n factores. Parra ello, usaremos la mediana de una dimensión, y tranformaremos el valor de un factor en 1, si este valor es mayor que la mediana de dicho factor; y en 0 si es menor o igual.

## Seleccionar usuario y película

Seleccionamos el usuario y la película que vamos a recomendar. 

In [2]:
user = 6
k_movies = 10

In [3]:
# Cargamos la matriz q_u del usuario
path = 'data/matrices_data/q_user_' + str(user) + '.csv'
q_u_DF = pd.read_csv(path)
q_u_DF['predictedRating'] = q_u_DF.apply(lambda row: sum(row[1:]), axis=1)
q_u_DF.sort_values(by='predictedRating', ascending=False)['movieId'][:k_movies].values

array([ 1089,   593,   367,  5349,   541,   316,  2959,   480,  1206,
       33794], dtype=int64)

In [4]:
movieId_to_recommend = 1089

In [5]:
matrix_qu_DF = q_u_DF.drop(columns=['predictedRating'])
matrix_qu_DF

,movieId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,0.301498,0.467121,0.141795,0.017201,0.217677,0.113700,0.105127,0.015963,0.374883,0.636164,0.163629,0.055172,0.024918,0.106678,0.021532
1,10,0.173557,0.777252,0.221952,0.006801,0.022617,0.175456,0.173607,0.254761,0.234166,0.009610,0.020329,0.475115,0.174280,0.267083,0.498224
2,32,0.094460,2.032427,0.044375,0.004282,0.144660,0.023100,0.397514,0.178309,0.225353,0.305960,0.051185,0.187642,0.137801,0.256530,0.458292
3,34,0.204817,0.158587,0.258000,0.000268,0.444853,0.010501,0.450876,0.064775,0.292796,0.506253,0.049599,0.980147,0.090821,0.343645,0.363801
4,47,0.154170,0.365045,0.202591,0.015639,0.054762,0.212742,0.359866,0.177833,0.133097,0.482720,0.119245,0.362057,0.142503,0.259136,0.253063
5,50,0.099826,0.632035,0.136374,0.010357,0.197646,0.197057,0.426205,0.157001,0.368861,0.330367,0.057201,0.868190,0.044336,0.200969,0.256748
6,110,0.061812,0.993951,0.112988,0.006016,0.288115,0.050773,0.395787,0.100468,0.278246,0.161413,0.214034,0.669061,0.088709,0.203029,0.386733
7,150,0.120984,0.805116,0.109486,0.004386,0.435585,0.172911,0.324352,0.024755,0.077976,0.131025,0.073994,0.108601,0.060365,0.061598,0.139414
8,153,0.132188,0.775298,0.070272,0.005150,0.318401,0.139703,0.128632,0.023701,0.095416,0.236839,0.232094,0.007535,0.017709,0.162594,0.514219
9,165,0.228425,1.202664,0.187897,0.000456,0.039653,0.117081,0.259150,0.037635,0.234474,0.348530,0.096835,0.035427,0.038541,0.096929,0.386874


## Cálculo de la mediana para cada dimensión
Para la matriz $Q_u$ del usuario al que hemos recomendado, calculamos la mediana para cada dimensión de la matriz. Guardamos esos valores en una lista. La posición de cada mediana en la lista será la misma que la posición de la dimensión en la matriz

In [6]:
# Calculo las medianas para cada dimension de la matriz qu (para cada columna)
dimensions = matrix_qu_DF.drop(columns=['movieId']).columns
medianas = [statistics.median(matrix_qu_DF[dimension].tolist()) for dimension in dimensions]
medianas 

[0.15161147319491364,
 0.9574825343237231,
 0.1119656394357922,
 0.007399577915053515,
 0.2538874168975497,
 0.16448402491580516,
 0.32456284389339674,
 0.12805768109245602,
 0.1936548391861197,
 0.39523801650012363,
 0.15350722409633294,
 0.41945989672264467,
 0.09142052169370156,
 0.20334400335419525,
 0.36585129671900934]

## Creación de la matriz binaria
En la matriz $Q_u$, sustituímos los valores de cada celda de la matriz dependiende del valor de la mediana de la dimensión (columna a la que pertenezca):
- Si el valor de la celda es mayor que el valor de la mediana de esa dimensión, le asignamos un 1.
- Si el valor de la celda es menor que el valor de la mediana de esa dimensión, le asignamos un 0.

In [7]:
for dimension in dimensions:
    matrix_qu_DF[dimension] = (matrix_qu_DF[dimension] > medianas[int(dimension)]).astype(int)

matrix_qu_DF

,movieId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0
1,10,1,0,1,0,0,1,0,1,1,0,0,1,1,1,1
2,32,0,1,0,0,0,0,1,1,1,0,0,0,1,1,1
3,34,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0
4,47,1,0,1,1,0,1,1,1,0,1,0,0,1,1,0
5,50,0,0,1,1,0,1,1,1,1,0,0,1,0,0,0
6,110,0,1,1,0,1,0,1,0,1,0,1,1,0,0,1
7,150,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
8,153,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
9,165,1,1,1,0,0,0,0,0,1,0,0,0,0,0,1


## Calculamos la matriz de similitud de películas

Usando la función de similitud `cosine`, `euclidean` o `manhattan`, calculamos la matriz de similitud de todas las películas entre ellas. Para ello usaremos el vector dentro de $Q_u$ que tiene cada película. Mostramos la matriz de similitudes en un mapa de calor para saber la dispersión de las similitudes.

In [8]:
similarity_function = 'cosine'

In [9]:
#matrix_qu_DF = q_u_DF.drop(columns=['movieId', 'predictedRating'])
items_sim_DF = pd.DataFrame(metrics.pairwise_distances(matrix_qu_DF, metric='cosine'))
items_sim_DF.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,0.529368,0.611550,0.579023,0.590701,0.599917,0.611856,0.622052,0.619590,0.615199,...,0.621846,0.621917,0.621920,0.621871,0.621824,0.621882,0.621867,0.622002,0.622023,0.622016
1,0.529368,0.000000,0.030516,0.031434,0.031916,0.033947,0.039008,0.041578,0.041609,0.039940,...,0.042062,0.042129,0.042130,0.042132,0.042093,0.042122,0.042131,0.042168,0.042167,0.042169
2,0.611550,0.030516,0.000000,0.004093,0.002779,0.002927,0.002598,0.003445,0.003261,0.002926,...,0.003380,0.003381,0.003391,0.003382,0.003388,0.003388,0.003387,0.003398,0.003398,0.003400
3,0.579023,0.031434,0.004093,0.000000,0.002354,0.002493,0.002440,0.003291,0.003315,0.003001,...,0.003423,0.003429,0.003424,0.003425,0.003430,0.003434,0.003429,0.003439,0.003441,0.003441
4,0.590701,0.031916,0.002779,0.002354,0.000000,0.001305,0.001975,0.001934,0.002095,0.001865,...,0.002010,0.002018,0.002018,0.002019,0.002013,0.002022,0.002017,0.002029,0.002029,0.002030


## Recuperar las K películas más similares

Recuperamos, a partir de esa matriz de similitudes, las K películas más similares a la película recomendada.

In [11]:
movies_SE = q_u_DF['movieId']
movie_index = movies_SE[movies_SE == movieId_to_recommend].index[0]
most_similar_items_index = items_sim_DF[movie_index].sort_values(ascending=False)[:k_movies].index
most_similar_items = movies_SE[most_similar_items_index.values].values

In [12]:
most_similar_items

array([  1,  10,  32,  34,  47,  50, 110, 165, 185, 208], dtype=int64)

## Mostrar retículo

Mostramos el retículo usando la técnica **FCA** de la pelicula a recomendar, junto con las K películas más similares.

In [13]:
matrix_qu_DF.rename(index=str, columns={"movieId": "id", "0": "d0", "1": "d1", "2": "d2", "3": "d3", "4": "d4", "5": "d5", "6": "d6", "7": "d7", "8": "d8", "9": "d9", "10": "d10", "11": "d11", "12": "d12", "13": "d13", "14": "d14", "15": "d15"}, inplace = True)
matrix_qu_DF.to_csv("data/experiment_data/movies_binary_dimensions.csv", index=False)
matrix_qu_DF

,id,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14
0,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0
1,10,1,0,1,0,0,1,0,1,1,0,0,1,1,1,1
2,32,0,1,0,0,0,0,1,1,1,0,0,0,1,1,1
3,34,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0
4,47,1,0,1,1,0,1,1,1,0,1,0,0,1,1,0
5,50,0,0,1,1,0,1,1,1,1,0,0,1,0,0,0
6,110,0,1,1,0,1,0,1,0,1,0,1,1,0,0,1
7,150,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
8,153,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
9,165,1,1,1,0,0,0,0,0,1,0,0,0,0,0,1


In [14]:
fca.calcular_reticulo('data/experiment_data/movies_binary_dimensions.csv', movieId_to_recommend, most_similar_items)